# Match Meise Collectors to Wikidata Items Using *Cosine Similarity*, `eventDate` Involved

In this example we add `eventDate` of the source data, when the sample/occurrence was collected, to have a time reference, when the collector should have been  alive.

Basically we …
- match of `canonical_string` of WikiData to `canonical_string` of the source collectors (abbreviated names and full names, if given), and
- parse collector source names beforehand to get individual names out of name lists in the source data, we have used <https://libraries.io/rubygems/dwc_agent>, and in general we
- follow the example of Niels Klazenga <https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb>
- calculate the name similarity and also a similarity score for the matching of life span with the (sampling) `eventDate`, and finally we
- write the output to provide a DarwinCore attribution structure (for `verbatimName` we would need the `source_data` name(s))

Technical Notes — Review Code perhaps:
- TODO review score calculation of the matching of relating eventData with range of yob, yod
- TODO review DwC agent output, keep at this time custom columns for filter-sort-evaluation convenience
- (NN ⇌ Cosine) refactor relation: wd_matchtest ⇌ wikidata_unique (replaced wd_matchtest → wikidata_unique)

### Load Wikidata Data Set

Construct data using Jupyter Notebook [create_wikidata_datasets_botanists-altlabel.ipynb](./create_wikidata_datasets_botanists-altlabel.ipynb)

Out of the Wikidata items data set we create a data frame with unique canonical name strings and their counts.

In [1]:
import pandas as pd
import pprint, time, os

explain_and_show_the_data = True
this_timestamp_for_data=20260210
# this_timestamp_for_data=time.strftime('%Y%m%d')

wikidata = pd.read_csv(
    # "data/wikidata_persons_botanists_20231030_1539.csv", # inverse match: [particle +] family, given
    # "data/wikidata_persons_botanists_20231116.csv",        # match: given [+ particle] + family[+ , suffix]
    # "data/wikidata_persons_botanists_20240312.csv", # with itemLabel + altLabel wyb, wye removed
    "data/wikidata_persons_botanists_20260210.csv", 
    index_col=0, low_memory=False,
    dtype={
        'yob':'Int32',
        'yod':'Int32',
        'wyb':'Int32',
        'wye':'Int32'
    }    
)
# # # # # # # # # # # # # # # # 
# def convert_to_datetime(year):
#     year = int(year)
#     days = int((year - int(year)) * 365.25)
#     base_date = pd.to_datetime(f'{year}-01-01', errors="ignore")
#     # ,format='%Y-%m-%d', errors='coerce'
#     # base_date = pd.Period(year, freq='D')
#     return base_date + pd.DateOffset(days=days)
# 
# def convert_to_datetime(year):
#     base_date = pd.to_datetime(f'{year}-01-01', errors="ignore")
# print(wikidata.dtypes)

# wikidata['yob_converted'] = wikidata['yob'].apply(convert_to_datetime)
# wikidata['yod_converted'] = wikidata['yod'].apply(lambda x: pd.Period(x, freq='Y')) # Given date string "-286" not likely a datetime
# # # # # # # # # # # # # # # # 

if explain_and_show_the_data:
    pprint.pprint(wikidata.columns)
    display(wikidata.head())

Index(['item', 'itemLabel', 'surname', 'initials', 'canonical_string',
       'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni',
       'abbr', 'bionomia_id', 'yob', 'yod', 'wikidata_link', 'orcid_link',
       'harv_link', 'ipni_link', 'bionomia_link'],
      dtype='str')


,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,ipni,abbr,bionomia_id,yob,yod,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q100142069,Frida Eggens,NaN,NaN,Eggens,Eggens,NaN,NaN,NaN,NaN,20045232-1,Eggens,NaN,<NA>,<NA>,http://www.wikidata.org/wiki/Q100142069,NaN,NaN,https://www.ipni.org/a/20045232-1,NaN
1,http://www.wikidata.org/entity/Q100142069,Frida Eggens,Frida,F.,F. Eggens,Frida Eggens,NaN,NaN,NaN,NaN,20045232-1,Eggens,NaN,<NA>,<NA>,http://www.wikidata.org/wiki/Q100142069,NaN,NaN,https://www.ipni.org/a/20045232-1,NaN
2,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,Elizabeth,E.,E. Harrison,Elizabeth Harrison,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795
3,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,NaN,NaN,Mrs A. H.,Mrs A. H.,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795
4,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,Mrs Arnold,M. A.,M. A. Harrison,Mrs Arnold Harrison,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795


In [2]:
# compile data having only unique canonical strings
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

if explain_and_show_the_data:
    display(wd_matchtest)
    display(wd_matchtest_fullnames)

,canonical_string,item
,,count
0,"""F."" Ryser",1
1,"""N.A. Antipova"" (lapsus)",1
2,"""N.A.Antipova"" (lapsus)",1
3,"""The grandmother of female scientists in Ghana""",1
4,"""Н. А. Антипова"" (lapsus)",1
...,...,...
171443,赵云鹏,1
171444,郭亚龙,1
171445,金井弘夫(Hiroo Kanai),1


,canonical_string_fullname,item
,,count
0,"""Fritz"" Ryser",1
1,"""N.A. Antipova"" (lapsus)",1
2,"""N.A.Antipova"" (lapsus)",1
3,"""The grandmother of female scientists in Ghana""",1
4,"""Н. А. Антипова"" (lapsus)",1
...,...,...
204788,赵云鹏,1
204789,郭亚龙,1
204790,金井弘夫(Hiroo Kanai),1


## Load Collectors Data Set

**Data sources:**

- Jupyter Notebook for [create_meise_gbif-occurrence_collectors_eventDate_dataset.ipynb](./create_meise_gbif-occurrence_collectors_eventDate_dataset.ipynb)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`

Technical notes:

- the corresponding objects, variable names of Nils’ python code were:
```
refactor df_avh = → = collectors
refactor df_avh['label'] = → = collectors['canonical_string_collector_parsed']
…
```

In [3]:
# unique names parsed already by ruby gem package: dwcagent

collectors = pd.read_csv(
    "data/Meise_doi-10.15468-dl.ax9zkh/occurrence_recordedBy_eventDate_occurrenceIDs_20230830_parsed.tsv",
    sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.

# Out of bounds nanosecond timestamp: 1652-01-01T00:00:00
#  because date nanoseconds range limitations of pandas, see https://stackoverflow.com/a/69507200/1240387
#  work around: use datetime or using pd.Periode(…)
print("modify time using pd.Periode(…) to get it work also on very old dates...")
for col in ['eventDate_mean', 'eventDate_min', 'eventDate_max']:
    print("- convert", col, "to pd.Period(...) in collectors")
    collectors[col] = collectors[col].apply(
        lambda x: pd.Period(
            x, freq='s' if col.lower().endswith('mean') else 'D' # Seconds or Day level
        )
    ) # D=day level
    # see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-period-aliases
print("modifing done.")

collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
if explain_and_show_the_data: display(collectors)

modify time using pd.Periode(…) to get it work also on very old dates...
- convert eventDate_mean to pd.Period(...) in collectors
- convert eventDate_min to pd.Period(...) in collectors
- convert eventDate_max to pd.Period(...) in collectors
modifing done.


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
43638,A,H. M. Llo,NaN,NaN,NaN,NaN,NaN,NaN,H.M. L[lo][???]a,parsed:H.M.Llo a,cleaned:H. M. Llo A,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
44218,A,Hameln,NaN,NaN,NaN,NaN,NaN,NaN,Hameln [a/d Weser],parsed:Hameln a<SEP>d Weser,cleaned:Hameln A<SEP>D Weser,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
84527,A,R.,NaN,NaN,NaN,NaN,NaN,NaN,R. Decha[] & R. []a,parsed:R. Decha<SEP>R. a,cleaned:R. Decha<SEP>R. A,1,http://www.botanicalcollections.be/specimen/BR...,1988-11-19 00:00:00,1988-11-19,1988-11-19
18419,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Church, A.C., Ismail, Ruskandi, A",parsed:A.C. Church<SEP>Ruskandi Ismail<SEP>A,cleaned:A.C. Church<SEP>Ruskandi Ismail<SEP>A,1,http://www.botanicalcollections.be/specimen/BR...,1995-10-26 00:00:00,1995-10-26,1995-10-26
87725,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Raynal R. & A,parsed:R. Raynal<SEP>A,cleaned:R. Raynal<SEP>A,1,http://www.botanicalcollections.be/specimen/BR...,1978-06-08 00:00:00,1978-06-08,1978-06-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24944,Žertová,A.,NaN,NaN,NaN,NaN,NaN,Dr.,Dr. A. Žertová,parsed:A. Žertová,cleaned:A. Žertová,6,http://www.botanicalcollections.be/specimen/BR...,1959-01-01 04:00:00,1958-08-13,1960-06-23
58954,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Klášterský & Žertová,parsed:Klášterský<SEP>Žertová,cleaned:Klášterský<SEP>Žertová,1,http://www.botanicalcollections.be/specimen/BR...,1958-05-20 00:00:00,1958-05-20,1958-05-20
91690,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,"S.P. Pinnerup, B. Ǿllgaard & L. Holm-Nielsen",parsed:S.P. Pinnerup<SEP>B. Ǿllgaard<SEP>L. Ho...,cleaned:S.P. Pinnerup<SEP>B. Ǿllgaard<SEP>L. H...,1,http://www.botanicalcollections.be/specimen/BR...,1975-08-29 00:00:00,1975-08-29,1975-08-29
118434,Т. Королева. V. Petrovsky,В.Пертовский,NaN,и,NaN,NaN,NaN,NaN,В. Пертовский и Т. Королева. V. Petrovsky & T....,parsed:В.Пертовский и Т. Королева. V. Petrovsk...,cleaned:В.Пертовский и Т. Королева. V. Petrovs...,1,http://www.botanicalcollections.be/specimen/BR...,1973-06-24 00:00:00,1973-06-24,1973-06-24


### Check Composition of Parsed Collector Data

In [4]:
# TODO review code of abbreviated names and full name matching
if explain_and_show_the_data: 
    criterion_fullnames = collectors.given.str.contains('^\\w{3,}', na=False)
    print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
    display(collectors[criterion_fullnames])

Show collecors given name has (propably) a full name (18092 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
44218,A,Hameln,NaN,NaN,NaN,NaN,NaN,NaN,Hameln [a/d Weser],parsed:Hameln a<SEP>d Weser,cleaned:Hameln A<SEP>D Weser,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
64323,A. Brabant,Lochenies,NaN,in,NaN,NaN,NaN,NaN,Lochenies in A. Brabant,parsed:Lochenies in A. Brabant,cleaned:Lochenies in A. Brabant,1,http://www.botanicalcollections.be/specimen/BR...,1892-01-01 00:00:00,1892-01-01,1892-01-01
34284,A. Chevalier,Fleury,NaN,in,NaN,NaN,NaN,NaN,Fleury in A. Chevalier,parsed:Fleury in A. Chevalier,cleaned:Fleury in A. Chevalier,2,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
22272,A. Cogniaux,Dandois,NaN,in,NaN,NaN,NaN,NaN,Dandois in A. Cogniaux,parsed:Dandois in A. Cogniaux,cleaned:Dandois in A. Cogniaux,1,http://www.botanicalcollections.be/specimen/BR...,1866-06-01 00:00:00,1866-06-01,1866-06-01
39898,A. Cogniaux,Gilbert,NaN,in,NaN,NaN,NaN,NaN,Gilbert in A. Cogniaux,parsed:Gilbert in A. Cogniaux,cleaned:Gilbert in A. Cogniaux,2,http://www.botanicalcollections.be/specimen/BR...,1864-08-06 00:00:00,1864-08-06,1864-08-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95208,Čermak,Stan,NaN,NaN,NaN,NaN,NaN,NaN,Stan. Čermak,parsed:Stan Čermak,cleaned:Stan Čermak,1,http://www.botanicalcollections.be/specimen/BR...,1926-05-01 00:00:00,1926-05-01,1926-05-01
95203,Čermák,Stan,NaN,NaN,NaN,NaN,NaN,NaN,Stan Čermák,parsed:Stan Čermák,cleaned:Stan Čermák,1,http://www.botanicalcollections.be/specimen/BR...,1925-10-12 00:00:00,1925-10-12,1925-10-12
97410,Şes-Frăţilescu,Tatiana,NaN,NaN,NaN,NaN,NaN,NaN,Tatiana Şes[]-Frăţilescu,parsed:Tatiana Şes-Frăţilescu,cleaned:Tatiana Şes-Frăţilescu,1,http://www.botanicalcollections.be/specimen/BR...,1972-06-13 00:00:00,1972-06-13,1972-06-13
97411,Şesan-Frăţilescu,Tatiana,NaN,NaN,NaN,NaN,NaN,NaN,Tatiana Şesan-Frăţilescu,parsed:Tatiana Şesan-Frăţilescu,cleaned:Tatiana Şesan-Frăţilescu,9,http://www.botanicalcollections.be/specimen/BR...,1972-04-06 08:00:00,1971-07-12,1973-06-12


In [5]:
# check the name-parsed columns if they are empty or need to be considerd as data for matching or not
if explain_and_show_the_data: 
    for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
        test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
        print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
        display(test_collectors.head().get(["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title"]))    


----------------------------------------
show names with **particle** found 8197 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
12654,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN
69635,A. Anspach,L.,NaN,in,NaN,NaN,NaN,NaN
64323,A. Brabant,Lochenies,NaN,in,NaN,NaN,NaN,NaN
34284,A. Chevalier,Fleury,NaN,in,NaN,NaN,NaN,NaN
49472,A. Chrková,J. Chrtek,NaN,at,NaN,NaN,NaN,NaN



----------------------------------------
show names with **suffix** found 188 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
35091,Almeda,Frank,Jr.,NaN,NaN,NaN,NaN,NaN
69174,Anderson,W.A.,Jr.,NaN,NaN,NaN,NaN,NaN
52875,Bailey,L.H.,Jr.,NaN,NaN,NaN,NaN,NaN
61663,Bailey,L.H.,Jr.,NaN,NaN,NaN,NaN,NaN
35259,Barkley,Robert,II,NaN,NaN,NaN,NaN,NaN



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title



----------------------------------------
show names with **appellation** found 1127 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
34792,A. Hardy,Lebrun,NaN,in,NaN,NaN,Fr.,NaN
9305,A. Hardy,Lebrun,NaN,in,NaN,NaN,Fr,NaN
117481,Abbon,G.,NaN,NaN,NaN,NaN,fr.,NaN
109094,Ahlberg,NaN,NaN,NaN,NaN,NaN,Fr.,NaN
34736,Ahlfvengren,E.,NaN,NaN,NaN,NaN,Fr.,NaN


Compile `canonical_string…` for the collector data we will later match the WikiData names with:

In [6]:
if explain_and_show_the_data: print("combine parts of names similar to WikiData's given name labels")

# 1. Prepare canonical name with conditional spacing: add space/comma directly to the value …
given = collectors['given'].fillna('')
# .where(condition, value_if_true) ensures spaces only appear when data is present
part = collectors['particle'].where(collectors['particle'].isna(), " " + collectors['particle'])
fam  = collectors['family'].where(collectors['family'].isna(), " " + collectors['family'])
suff = collectors['suffix'].where(collectors['suffix'].isna(), ", " + collectors['suffix'])
# 2. Vectorized concatenation
collectors['canonical_string_collector_parsed'] = (
    given + part.fillna('') + fam.fillna('') + suff.fillna('')
).str.strip()
# Optional: Clean up multiple spaces if they exist in the source data
collectors['canonical_string_collector_parsed'] = (
    collectors['canonical_string_collector_parsed'].str.replace(r'\s+', ' ', regex=True)
)

criterion = collectors["particle"].str.contains("\\w+ \\w+", na=False)

if explain_and_show_the_data: 
    # display(collectors['canonical_string_collector_parsed'][criterion].head())
    display(collectors[['canonical_string_collector_parsed', 'particle']][criterion].drop_duplicates().head(10))

combine parts of names similar to WikiData's given name labels


,canonical_string_collector_parsed,particle
5111,A. coenen in A. Hardy,coenen in
1668,A. Hardy avec Mr. Fonsny in A. Hardy,avec Mr. Fonsny in
109856,H. Ponckier in L. Quadvlieg in A. Hardy,in L. Quadvlieg in
60163,L. Coututier in Simon in A. Hardy,in Simon in
62121,Laboulle de Verviers in A. Hardy,de Verviers in
63440,Lejeune in Sonnet in A. Hardy,in Sonnet in
51896,J. Van de Sande in A. Jans,Van de Sande in
53153,J.E. De Langhe in A. Jans,De Langhe in
56733,José Van Baelen in A. Lawalrée,Van Baelen in
37706,G.-C. van Haesendonck in A. Thielens,van Haesendonck in


In [7]:
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)

these_columns=["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title", 'canonical_string_collector_parsed']

if 'source_data' in collectors.columns:
    these_columns.append("source_data")

if explain_and_show_the_data: display(collectors.tail().get(these_columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data
24944,Žertová,A.,NaN,NaN,NaN,NaN,NaN,Dr.,A. Žertová,Dr. A. Žertová
58954,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Žertová,Klášterský & Žertová
91690,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,B. Ǿllgaard,"S.P. Pinnerup, B. Ǿllgaard & L. Holm-Nielsen"
118434,Т. Королева. V. Petrovsky,В.Пертовский,NaN,и,NaN,NaN,NaN,NaN,В.Пертовский и Т. Королева. V. Petrovsky,В. Пертовский и Т. Королева. V. Petrovsky & T....
52094,Ẅttewaall,J.,NaN,NaN,NaN,NaN,NaN,NaN,J. Ẅttewaall,J. [Ẅ]ttewaall


In [8]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    source_data=('source_data', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]), # custom function, to get the first entry
    collectors_eventDate_mean=('eventDate_mean', 'mean'),
    collectors_eventDate_min=('eventDate_min', 'min'),
    collectors_eventDate_max=('eventDate_max', 'max')
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)
collectors_unique.drop_duplicates(inplace=True)

if explain_and_show_the_data: display(collectors_unique)

# column naming perhaps more clear (because we condensed the data)?
# collectors=collectors.add_suffix('_namegrouped') \
#  if not any(col.endswith("_namegrouped") for col in list(collectors.columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,"Church, A.C., Ismail, Ruskandi, A",222,http://www.botanicalcollections.be/specimen/BR...,1941-11-06 11:24:04,1809-01-01,2000-07-14
1,Aux,A,NaN,NaN,NaN,NaN,NaN,NaN,A Aux,[]a[] []aux,3,http://www.botanicalcollections.be/specimen/BR...,1968-01-31 08:00:00,1967-06-01,1968-06-02
2,Funk,A,NaN,NaN,NaN,NaN,NaN,NaN,A Funk,a Funk,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
3,Isaac Holden,A,NaN,NaN,NaN,NaN,NaN,NaN,A Isaac Holden,a) Isaac Holden;b) W.A. Setchell,1,http://www.botanicalcollections.be/specimen/BR...,1892-09-01 00:00:00,1892-09-01,1892-09-01
4,Jablonski,A,NaN,NaN,NaN,NaN,NaN,NaN,A Jablonski,a Jablonski,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69215,Černev,Ž.,NaN,NaN,NaN,NaN,NaN,NaN,Ž. Černev,"T. Mešinev, Ž. Černev, C. Kacareva",1,http://www.botanicalcollections.be/specimen/BR...,1973-07-16 00:00:00,1973-07-16,1973-07-16
69216,Černeva,Ž.,NaN,NaN,NaN,NaN,NaN,NaN,Ž. Černeva,"Ž. Černeva, P. Gerginov",104,http://www.botanicalcollections.be/specimen/BR...,1975-11-15 09:21:13,1973-06-10,1979-09-20
69217,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Žertová,Klášterský & Žertová,1,http://www.botanicalcollections.be/specimen/BR...,1958-05-20 00:00:00,1958-05-20,1958-05-20
69218,Péterfi,ϯ M.,NaN,NaN,NaN,NaN,NaN,NaN,ϯ M. Péterfi,Ϯ M. Péterfi & M. Pr[]c[],1,http://www.botanicalcollections.be/specimen/BR...,1921-06-02 00:00:00,1921-06-02,1921-06-02


## Set Up the Cosine Similarity and Text Search

See 
- for the application code https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb
- for reading on the topic: Taylor, Josh. 2019. ‘Fuzzy Matching at Scale’. Towards Data Science (blog). 2 July 2019. https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536.

The `ngrams`-function is used as an analyzer in the text search later.

In [9]:
import pandas as pd, numpy as np, re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sparse_dot_topn import sp_matmul_topn # pip install sparse-dot-topn

def get_matches_df(sparse_matrix, A, B, top=100):
    """
    Get the matches of a data frame

    @param sparse_matrix: the matching matrix to apply
    @param A: the query data
    @param B: the match data
    @return: dataframe
    """
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similarity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = A[sparserows[index]]
        right_side[index] = B[sparsecols[index]]
        similarity[index] = round(sparse_matrix.data[index], 3)

    return pd.DataFrame({'namematch_source_data': left_side,
                         'namematch_resource_data': right_side,
                         'namematch_similarity': similarity})

!pip install ftfy
from ftfy import fix_text
import unicodedata

# Regex for unwanted special characters
CHARS_TO_REMOVE_RX = re.compile(r"[)(.|\[\]{}'„“”\"‚‘’›‹»«]|[,-./]|\sBD")
MULTIPLE_SPACES_RX = re.compile(r' +')

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text
     
    @param string: the text string to perform the ngram splitting on 
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    if not string:
        return []

    # (fix encoding errors)
    string = fix_text(string)

    # Normalization (IMPORTANT for Unicode)
    # NFC ensures that characters such as ‘é’ are treated as a single character
    # rather than as a combination of ‘e’ + accent.
    string = unicodedata.normalize('NFC', string)
    string = string.lower()    
    # (other languages often use different separators; kept generic here)
    string = string.replace('&', 'and').replace(',', ' ').replace('-', ' ')    
    # character cleanup but leave then as spaces
    string = CHARS_TO_REMOVE_RX.sub(' ', string)    
    # Normalization of spaces & case: ….title() also works with umlauts (e.g., “öllegard” -> “Öllegard”)
    string = string.title()
    string = MULTIPLE_SPACES_RX.sub(' ', string).strip()    
    # Padding & N-gram generation
    string = f" {string} "
    return [string[i : i + n] for i in range(len(string) - n + 1)]

def calculateTFIDFmatchingOfData(query_data, match_data, cossim_ntop=1, cossim_lower_bound=0.5):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with awesome_cossim_topn() and return matched data

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param cossim_ntop: how many cossim matches each shall be calculated (default 1, i.e. the highest similarity) — increase it to get more alternative
        matches with less similarity
    @param cossim_lower_bound: where is the lower similarity cut off to regard data as similar (default 0.5)

    @requires get_get_matches_df()
    @requires ngrams()
    @requires awesome_cossim_topn()
    @requires TfidfVectorizer()

    @return: a data frame dictionary: namematch_source_data, namematch_resource_data, namematch_similarity (from @see get_matches_df())
    @rtype pd.DataFrame
    """

    import time
    time_start = time.time()

    # Vectorize Wikidata name (use fit_transform())
    print('Vectorizing data. This may take a while...')
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
    tf_idf_matrix_clean = vectorizer.fit_transform(match_data)
    # Vectorize collectors’ names (use transform())
    tf_idf_matrix_dirty = vectorizer.transform(query_data)

    duration = time.time() - time_start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    # Calculate Cosine Similarity; keep only the best match (ntop=1) and only if the similarity is greater than 0.5 (lower_bound=0.5)
    # (lower_bound: a threshold that the element of A*B must be greater than
    #  https://github.com/ing-bank/sparse_dot_topn/blob/3f40611b0553b50c27f23c7dcffc3ca9a9e8f5b5/sparse_dot_topn/awesome_cossim_topn.py#L26C9-L26C78)
    cossim_matches = sp_matmul_topn( # former function was awesome_cossim_topn()
        tf_idf_matrix_dirty,
        tf_idf_matrix_clean.transpose(),
        top_n=cossim_ntop,
        threshold=cossim_lower_bound, sort=True
    )
    print("Cossim matches calculated after %s s" % (time.time() - time_start))

    print("Get all matches together ...")
    # construct the matching data frame
    matches_df = get_matches_df(
        cossim_matches,
        query_data,
        match_data,
        top=0
    )
    print("Done. Matches calculated after %s s" % (time.time() - time_start))

    return matches_df


In [11]:
# some example data
samples = [
    ("simple name", "Klazenga, N."),
    ("simple name", "金井弘夫(Hiroo Kanai)"),
    ("simple name", "Н. А. Антипова"),
    ("data from collectors", collectors_unique["canonical_string_collector_parsed"].at[1]),
    ("data from match-test", collectors_unique["canonical_string_collector_parsed"].at[1]),
    ("data from match-test, full names", wd_matchtest_fullnames['canonical_string_fullname'].at[0])
]

print("Show ngram examples:")
for label, name in samples:
    print(f"- {label} “{name}”:", ngrams(name))

# some example data
print('\n(WikiData’s) canonical_string = (constructed) canonical_string_fullname:')
# We only take the first 5 lines of both columns.
short_names = wd_matchtest['canonical_string'].head(5)
long_names = wd_matchtest_fullnames['canonical_string_fullname'].head(5)

for short, long in zip(short_names, long_names):
    print(f"- {short} = {long}")


Show ngram examples:
- simple name “Klazenga, N.”: [' Kl', 'Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N', ' N ']
- simple name “金井弘夫(Hiroo Kanai)”: [' 金井', '金井弘', '井弘夫', '弘夫 ', '夫 H', ' Hi', 'Hir', 'iro', 'roo', 'oo ', 'o K', ' Ka', 'Kan', 'ana', 'nai', 'ai ']
- simple name “Н. А. Антипова”: [' Н ', 'Н А', ' А ', 'А А', ' Ан', 'Ант', 'нти', 'тип', 'ипо', 'пов', 'ова', 'ва ']
- data from collectors “A Aux”: [' A ', 'A A', ' Au', 'Aux', 'ux ']
- data from match-test “A Aux”: [' A ', 'A A', ' Au', 'Aux', 'ux ']
- data from match-test, full names “"Fritz" Ryser”: [' Fr', 'Fri', 'rit', 'itz', 'tz ', 'z R', ' Ry', 'Rys', 'yse', 'ser', 'er ']

(WikiData’s) canonical_string = (constructed) canonical_string_fullname:
- "F." Ryser = "Fritz" Ryser
- "N.A. Antipova" (lapsus) = "N.A. Antipova" (lapsus)
- "N.A.Antipova" (lapsus) = "N.A.Antipova" (lapsus)
- "The grandmother of female scientists in Ghana" = "The grandmother of female scientists in Ghana"
- "Н. А. Антипова" (lapsus) = "Н. А. А

## Perform the Matching

In [12]:
criterion_fullnames = collectors_unique.given.str.contains('^\\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values

print("Calculate rather the abbreviated names only …")
matches = calculateTFIDFmatchingOfData(
    collectors_names, 
    wd_matchtest['canonical_string'], 
    cossim_ntop=1 # e.g. cossim_ntop=3 would give more alternative matches as well, having lower similarities, data would increase 3 times as well
)
matches = matches.sort_values(by=['namematch_similarity'], ascending=[False])
matches = matches.reset_index(names=['old_index'])
if explain_and_show_the_data: display(matches)

Calculate rather the abbreviated names only …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 5.637071847915649 s
Cossim matches calculated after 12.078066110610962 s
Get all matches together ...
Done. Matches calculated after 12.325722455978394 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,45299,ϯ M. Péterfi,M. Péterfi,1.0
1,45280,Ö. Szatala,Ö. Szatala,1.0
2,45271,É. Marchal,É. Marchal,1.0
3,45270,É. Laurent,É. Laurent,1.0
4,45266,É. Chas,É. Chas,1.0
...,...,...,...,...
45295,36317,R. Mbuwa,Kibuwa,0.5
45296,22282,J. Rovátes,J. Rova,0.5
45297,10785,E. Bouckenaere,Walckenaer,0.5
45298,26847,L.C. Okeke,Oken,0.5


In [13]:
# criterion_fullnames = collectors_unique.given.str.contains('^\\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
print("Calculate rather the full names only …")
matches_fullnames = calculateTFIDFmatchingOfData(
    collectors_fullnames, 
    wd_matchtest_fullnames['canonical_string_fullname'], 
    cossim_ntop=1 # 10 would give more alternative matches also with lesser similarity
)

matches_fullnames = matches_fullnames.sort_values(by=['namematch_similarity'], ascending=[False])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

if explain_and_show_the_data: display(matches_fullnames)

Calculate rather the full names only …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 5.066992282867432 s
Cossim matches calculated after 8.20695972442627 s
Get all matches together ...
Done. Matches calculated after 8.288578510284424 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,6707,Melany H. Chapin,Melany H. Chapin,1.0
1,6703,Melaku Wondafrash,Melaku Wondafrash,1.0
2,9957,Yea-Chen Liu,Yea Chen Liu,1.0
3,9981,Yves Issembé,Yves Issembé,1.0
4,9975,Yukinori Yamamoto,Yukinori Yamamoto,1.0
...,...,...,...,...
10033,1939,Daniel B. Plyler,Daniel,0.5
10034,6463,Marielle Fauchaux,Louis Marie Faucheron,0.5
10035,144,Afzal Chaudhy,Afzal-Rafii,0.5
10036,8857,Soutié in F. Crépin,François Crépin,0.5


## Create Output Results

Combine the matches data frame back to the (Meise) collectors and Wikidata items …

Note: merging 18.770.000 collector matches earlier to wikidata was too much to calculate. Hence the descision was to make the data unique by canonical_string_collector_parsed.

In [14]:
if explain_and_show_the_data: print("join matches data frame back to source collectors dataframe")
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data', 
    how='left'
)

collectors_matches.dropna(subset=['namematch_similarity'], inplace=True) # drop all matches that the method gave a NA (we narrowed the similarity range from 0.5 to 1.0)
if explain_and_show_the_data:
    print("show match results of mostly abbreviated names")
    display(collectors_matches) # 17552 rows × 15 columns

join matches data frame back to source collectors dataframe
show match results of mostly abbreviated names


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,"Church, A.C., Ismail, Ruskandi, A",222,http://www.botanicalcollections.be/specimen/BR...,1941-11-06 11:24:04,1809-01-01,2000-07-14,0.0,A,A. A.,0.730
2,Funk,A,NaN,NaN,NaN,NaN,NaN,NaN,A Funk,a Funk,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,1.0,A Funk,A.Funk,1.000
3,Isaac Holden,A,NaN,NaN,NaN,NaN,NaN,NaN,A Isaac Holden,a) Isaac Holden;b) W.A. Setchell,1,http://www.botanicalcollections.be/specimen/BR...,1892-09-01 00:00:00,1892-09-01,1892-09-01,2.0,A Isaac Holden,Isaac,0.708
4,Jablonski,A,NaN,NaN,NaN,NaN,NaN,NaN,A Jablonski,a Jablonski,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,3.0,A Jablonski,E. Jablonski,0.901
5,Kuhlew,A,NaN,NaN,NaN,NaN,NaN,Dr.,A Kuhlew,Dr. a Kuhlew.,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,4.0,A Kuhlew,Kuhlenk.,0.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69212,Cerneva,Ž.,NaN,NaN,NaN,NaN,NaN,NaN,Ž. Cerneva,"M. Markova, Ž. Cerneva, P. Gerginov",1,http://www.botanicalcollections.be/specimen/BR...,1978-05-24 00:00:00,1978-05-24,1978-05-24,45295.0,Ž. Cerneva,O. Cherneva,0.507
69215,Černev,Ž.,NaN,NaN,NaN,NaN,NaN,NaN,Ž. Černev,"T. Mešinev, Ž. Černev, C. Kacareva",1,http://www.botanicalcollections.be/specimen/BR...,1973-07-16 00:00:00,1973-07-16,1973-07-16,45296.0,Ž. Černev,Čern.,0.541
69216,Černeva,Ž.,NaN,NaN,NaN,NaN,NaN,NaN,Ž. Černeva,"Ž. Černeva, P. Gerginov",104,http://www.botanicalcollections.be/specimen/BR...,1975-11-15 09:21:13,1973-06-10,1979-09-20,45297.0,Ž. Černeva,Čern.,0.527
69217,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Žertová,Klášterský & Žertová,1,http://www.botanicalcollections.be/specimen/BR...,1958-05-20 00:00:00,1958-05-20,1958-05-20,45298.0,Žertová,A. Chrtková-Žertová,0.705


In [15]:
if explain_and_show_the_data: print("show full name matches, and append them to all matches")
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed' , right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

if explain_and_show_the_data:
    display(collectors_matches_fullname) # 429 rows × 15 columns

show full name matches, and append them to all matches


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,Bunge,AAv,NaN,NaN,NaN,NaN,NaN,NaN,AAv Bunge,Bunge A.A.v.,688,http://www.botanicalcollections.be/specimen/BR...,1843-04-27 13:11:38,1839-01-01,1898-05-01,0,AAv Bunge,Bunge.,0.624
1,Rudio,ABaun,NaN,NaN,NaN,NaN,NaN,NaN,ABaun Rudio,"Rudio, ABaun",1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,1,ABaun Rudio,Rudio,0.653
2,Schneller,AHardy,NaN,NaN,NaN,NaN,NaN,NaN,AHardy Schneller,"Schneller, AHardy",1,http://www.botanicalcollections.be/specimen/BR...,1868-07-01 00:00:00,1868-07-01,1868-07-01,2,AHardy Schneller,Schneller,0.658
3,Smit,Aad,NaN,NaN,NaN,NaN,NaN,NaN,Aad Smit,Aad Smit,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,3,Aad Smit,Smit,0.602
4,Cajander,Aarno,NaN,NaN,NaN,NaN,NaN,NaN,Aarno Cajander,Aarno Cajander,68,http://www.botanicalcollections.be/specimen/BR...,1929-09-27 19:38:10,1926-07-28,1934-08-05,4,Aarno Cajander,Aimo Aarno Antero Cajander,0.712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10033,Greg,von Dresden,NaN,NaN,NaN,NaN,NaN,NaN,von Dresden Greg,"Greg, von Dresden",1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,10033,von Dresden Greg,Greg.,0.504
10034,Pereira,Ângelo,NaN,NaN,NaN,NaN,NaN,NaN,Ângelo Pereira,"[M].F. Correia, A. Marques et Ângelo Pereira",1,http://www.botanicalcollections.be/specimen/BR...,1973-10-16 00:00:00,1973-10-16,1973-10-16,10034,Ângelo Pereira,Ângelo Pereira,1.000
10035,Persson,Åke,NaN,NaN,NaN,NaN,NaN,NaN,Åke Persson,Åke Persson,1,http://www.botanicalcollections.be/specimen/BR...,1950-08-14 00:00:00,1950-08-14,1950-08-14,10035,Åke Persson,Åke Åkerman,0.590
10036,Gérard,Éléonore,NaN,NaN,NaN,NaN,NaN,NaN,Éléonore Gérard,Éléonore Gérard,6,http://www.botanicalcollections.be/specimen/BR...,1879-10-29 04:00:00,1861-04-24,1886-08-24,10036,Éléonore Gérard,Gérard,0.591


In [16]:
# join all name matches together
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)
if explain_and_show_the_data:
    print("show match results of all abbreviated and full names")
    display(collectors_all_matches.head())

show match results of all abbreviated and full names


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
42454,A. García,M.,NaN,NaN,NaN,NaN,NaN,NaN,M. A. García,"Rafael Torres C., A. García M., L. Cortes",10,http://www.botanicalcollections.be/specimen/BR...,1987-04-09 19:12:00,1987-04-07,1987-04-13,28109.0,M. A. García,M.A.García,1.0
2670,A.A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A.A,A.A.,3,http://www.botanicalcollections.be/specimen/BR...,2013-12-23 08:00:00,2013-12-22,2013-12-24,2225.0,A.A,A. A.,1.0
2731,A.B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A.B,A.B.,96,http://www.botanicalcollections.be/specimen/BR...,1862-02-12 06:49:05,1827-05-01,1904-05-01,2281.0,A.B,A. B.,1.0
2819,A.C.B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A.C.B,?A.C.B.,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,2356.0,A.C.B,A C B,1.0
3434,A.R.-Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A.R.-Smith,"G. Pope, A.R.-Smith & D.Goyder",1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,2888.0,A.R.-Smith,A. R. Smith,1.0


Save the plain name matching results only ...

In [17]:
# if not os.path.exists('data'):
#     print("Make data directory for saving …")
#     os.makedirs('data')
# 
# # Set some global varialbes
# # this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
# this_timestamp_for_data=20231030
# 
# this_output_file='data/results_meise_collectors_vs_wikidata-botanists_cossim-similarity_plain-names_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_all_matches.to_csv(this_output_file)
# 
# print("Wrote plain name matches of collector names into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )

# old code # Join Wikidata items
# df_avh_matches_wikidata = pd.merge(df_avh_matches, df_wikidata                , left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata = pd.merge(df_avh_matches_wikidata, df_wikidata_unique, left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata.rename(columns={df_avh_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)


In [18]:
if explain_and_show_the_data: print("merge now the matching data and the wiki data’s on the conaonical string name")

collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

merge now the matching data and the wiki data’s on the conaonical string name


In [19]:
if explain_and_show_the_data:
    print("Show some name match examples (e.g. «Louis…» matching various names) …")
    for testname in ['Louis', 'Abbot']:
        criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].str.contains(testname)
        this_table=collectors_matches_g1_merged_wikidata[criterion].get([
            # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
            'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
            'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
            'itemLabel', 
            'canonical_string_fullname', # canonical_string_fullname contains the former itemMatchingLabel
            'wikidata_link',
            'collectors_eventDate_min', 'collectors_eventDate_max',
            'yob', 'yod',            
        ]).sort_values(by=['namematch_similarity'], ascending=[False])
        print("# ---------------------------------------------\n# «%s…» as test name, %d collector names begin with:" % (testname, criterion.sum()))    
        display(this_table)

Show some name match examples (e.g. «Louis…» matching various names) …
# ---------------------------------------------
# «Louis…» as test name, 185 collector names begin with:


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_similarity,itemLabel,canonical_string_fullname,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod
3433,2348,http://www.botanicalcollections.be/specimen/BR...,A.M. Louis,A.M.Louis,1.000,Adriaan M. Louis,A.M.Louis,http://www.wikidata.org/wiki/Q21338327,1902-11-28,1995-05-04,1944,<NA>
21818,4,http://www.botanicalcollections.be/specimen/BR...,H. Louis,H. Louis,1.000,Herbert Louis Mason,Herbert Louis,http://www.wikidata.org/wiki/Q11925452,1860-08-01,1965-09-01,1896,1994
33986,12,http://www.botanicalcollections.be/specimen/BR...,Louis,Louis,1.000,Jean Laurent Prosper Louis,Louis,http://www.wikidata.org/wiki/Q5928759,1878-06-01,1915-06-18,1903,1947
33992,9,http://www.botanicalcollections.be/specimen/BR...,Louis-Marie,Louis-Marie,1.000,Louis-Marie Lalonde,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1934-08-03,1952-07-01,1896,1978
59632,151,http://www.botanicalcollections.be/specimen/BR...,Jean-Louis De Sloover,Jean Louis De Sloover,1.000,Jean Louis J. A. De Sloover,Jean Louis De Sloover,http://www.wikidata.org/wiki/Q5928774,1943-10-31,1956-09-22,1936,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
43354,1,http://www.botanicalcollections.be/specimen/BR...,R. De Litardière in Louis Magnel,R. V. d. Litardière,0.507,René Verriet de Litardière,René Verriet de Litardière,http://www.wikidata.org/wiki/Q4263547,1920-06-14,1920-06-14,1888,1957
44409,9,http://www.botanicalcollections.be/specimen/BR...,R. de Litardière in Louis Magnel,R. V. d. Litardière,0.507,René Verriet de Litardière,René Verriet de Litardière,http://www.wikidata.org/wiki/Q4263547,1920-06-17,1922-06-18,1888,1957
1580,1,http://www.botanicalcollections.be/specimen/BR...,A. Marchal in Louis Magnel,Louis-Marie,0.503,Louis-Marie Lalonde,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1928-08-30,1928-08-30,1896,1978
15188,2,http://www.botanicalcollections.be/specimen/BR...,Ed Bernays Louis Magnel,E. Bernays,0.501,Edouard Bernays,Edouard Bernays,http://www.wikidata.org/wiki/Q48179652,1920-08-08,1923-07-22,1874,1940


# ---------------------------------------------
# «Abbot…» as test name, 13 collector names begin with:


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_similarity,itemLabel,canonical_string_fullname,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod
3728,1,http://www.botanicalcollections.be/specimen/BR...,A.T.D. Abbott,A. T. D. Abbott,1.000,A. T. D. Abbott,A. T. D. Abbott,http://www.wikidata.org/wiki/Q117328147,2010-01-07,2010-01-07,1936,2013
3848,2,http://www.botanicalcollections.be/specimen/BR...,Abbott,Abbott,1.000,George Abbott,Abbott,http://www.wikidata.org/wiki/Q47112598,1982-11-16,1982-11-16,<NA>,<NA>
27105,1,http://www.botanicalcollections.be/specimen/BR...,J. Richard Abbott,J.Richard Abbott,1.000,J. Richard Abbott,J.Richard Abbott,http://www.wikidata.org/wiki/Q18982386,1995-03-22,1995-03-22,1968,<NA>
43067,2,http://www.botanicalcollections.be/specimen/BR...,R. Abbott,R. Abbott,1.000,R.J. Abbott,Richard Abbott,http://www.wikidata.org/wiki/Q33660683,1995-01-27,1995-06-20,1945,2024
53036,1,http://www.botanicalcollections.be/specimen/BR...,W.L. Abbott,W. L. Abbott,1.000,William Louis Abbott,W. L. Abbott,http://www.wikidata.org/wiki/Q635604,1922-02-26,1922-02-26,1860,1936
53040,1,http://www.botanicalcollections.be/specimen/BR...,W.L. Abbott,W. L. Abbott,1.000,William Louis Abbott,William Louis Abbott,http://www.wikidata.org/wiki/Q635604,1922-02-26,1922-02-26,1860,1936
53039,1,http://www.botanicalcollections.be/specimen/BR...,W.L. Abbott,W. L. Abbott,1.000,William Louis Abbott,William L. Abbott,http://www.wikidata.org/wiki/Q635604,1922-02-26,1922-02-26,1860,1936
53038,1,http://www.botanicalcollections.be/specimen/BR...,W.L. Abbott,W. L. Abbott,1.000,William Louis Abbott,W. L. Abbott,http://www.wikidata.org/wiki/Q635604,1922-02-26,1922-02-26,1860,1936
53037,1,http://www.botanicalcollections.be/specimen/BR...,W.L. Abbott,W. L. Abbott,1.000,William Louis Abbott,W L Abbott,http://www.wikidata.org/wiki/Q635604,1922-02-26,1922-02-26,1860,1936
52767,1,http://www.botanicalcollections.be/specimen/BR...,W.C. Abbott,C. Abbott,0.923,Charles Abbot,Charles Abbott,http://www.wikidata.org/wiki/Q1063507,NaT,NaT,1761,1817


In [20]:
if explain_and_show_the_data:
    print("list all column names so far, and show example data")
    pprint.pprint(collectors_matches_g1_merged_wikidata.columns)
    display(collectors_matches_g1_merged_wikidata.head())

list all column names so far, and show example data
Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'source_data', 'occurrenceID_collectors_count',
       'occurrenceID_collectors_firstsample', 'collectors_eventDate_mean',
       'collectors_eventDate_min', 'collectors_eventDate_max', 'old_index',
       'namematch_source_data', 'namematch_resource_data',
       'namematch_similarity', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod',
       'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='str')


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,...,ipni,abbr,bionomia_id,yob,yod,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,"Church, A.C., Ismail, Ruskandi, A",...,NaN,NaN,Q264269,1799,1871,http://www.wikidata.org/wiki/Q264269,NaN,https://kiki.huh.harvard.edu/databases/botanis...,NaN,https://bionomia.net/Q264269
1,Funk,A,NaN,NaN,NaN,NaN,NaN,NaN,A Funk,a Funk,...,19045-1,A.Funk,Q21337490,1925,2010,http://www.wikidata.org/wiki/Q21337490,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/19045-1,https://bionomia.net/Q21337490
2,Isaac Holden,A,NaN,NaN,NaN,NaN,NaN,NaN,A Isaac Holden,a) Isaac Holden;b) W.A. Setchell,...,12643-1,F.M.Leight.,Q5864685,1909,2006,http://www.wikidata.org/wiki/Q5864685,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12643-1,https://bionomia.net/Q5864685
3,Jablonski,A,NaN,NaN,NaN,NaN,NaN,NaN,A Jablonski,a Jablonski,...,12575-1,Jabl.,Q3059563,1892,1975,http://www.wikidata.org/wiki/Q3059563,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12575-1,https://bionomia.net/Q3059563
4,Kuhlew,A,NaN,NaN,NaN,NaN,NaN,Dr.,A Kuhlew,Dr. a Kuhlew.,...,29926-1,Kuhlenk.,NaN,<NA>,<NA>,http://www.wikidata.org/wiki/Q36665275,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/29926-1,NaN


In [21]:
# Select useful columns for data results
show_display_output=explain_and_show_the_data
# show_display_output=False
if show_display_output:
    # Select useful columns for data results
    collectors_wikidata_cossimOrKmeans = collectors_matches_g1_merged_wikidata[
        ['canonical_string_collector_parsed', 'family', 'given', 
         'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
         'source_data',
        'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
        'item', 'canonical_string', 'itemLabel',
        'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 
         # 'wyb'
        ]
    ].copy()
    
    # Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
    collectors_wikidata_cossimOrKmeans.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)
    
    display(collectors_wikidata_cossimOrKmeans)

,canonical_string_collector_parsed,family,given,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,source_data,namematch_source_data,namematch_resource_data,namematch_similarity,item,...,itemLabel,orcid,viaf,isni,harv,ipni,abbr,bionomia_id,yob,yod
34196,M. A. García,A. García,M.,10,http://www.botanicalcollections.be/specimen/BR...,"Rafael Torres C., A. García M., L. Cortes",M. A. García,M.A.García,1.0,http://www.wikidata.org/entity/Q18984786,...,Miguel A. García,NaN,NaN,NaN,NaN,38416-1,M.A.García,NaN,<NA>,<NA>
2737,A.A,A.A,NaN,3,http://www.botanicalcollections.be/specimen/BR...,A.A.,A.A,A. A.,1.0,http://www.wikidata.org/entity/Q264269,...,Anna Atkins,NaN,25669621,0000000066812379,21686,NaN,NaN,Q264269,1799,1871
2813,A.B,A.B,NaN,96,http://www.botanicalcollections.be/specimen/BR...,A.B.,A.B,A. B.,1.0,http://www.wikidata.org/entity/Q21390161,...,Alexander Bierig,NaN,163555875,000000011228212X,NaN,NaN,NaN,Q21390161,1884,1963
2918,A.C.B,A.C.B,NaN,1,http://www.botanicalcollections.be/specimen/BR...,?A.C.B.,A.C.B,A C B,1.0,http://www.wikidata.org/entity/Q5650096,...,Alexander Curt Brade,NaN,296689217,0000000401949066,78512,1056-1,Brade,Q5650096,1881,1971
3639,A.R.-Smith,A.R.-Smith,NaN,1,http://www.botanicalcollections.be/specimen/BR...,"G. Pope, A.R.-Smith & D.Goyder",A.R.-Smith,A. R. Smith,1.0,http://www.wikidata.org/entity/Q4812446,...,Alan Reid Smith,NaN,2598973,0000000066507067,62311,9771-1,A.R.Sm.,NaN,1943,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27325,J. Stallaert,Stallaert,J.,1,http://www.botanicalcollections.be/specimen/BR...,J. Stallaert,J. Stallaert,L. J. Stal,0.5,http://www.wikidata.org/entity/Q5981892,...,Lucas J. Stal,0000-0002-2775-9653,5029257,0000000080843366,72767,30488-1,Stal,NaN,1953,<NA>
48763,Sudlitz,Sudlitz,NaN,1,http://www.botanicalcollections.be/specimen/BR...,[Sudlitz],Sudlitz,Seidlitz,0.5,http://www.wikidata.org/entity/Q4190175,...,Nicolai Karlovič (Karl Samuel) Seidlitz,NaN,32774680,0000000002320105,42445,9443-1,Seidlitz,Q4190175,1831,1907
50102,Thirseaux,Thirseaux,NaN,1,http://www.botanicalcollections.be/specimen/BR...,Thirseaux,Thirseaux,Damseaux,0.5,http://www.wikidata.org/entity/Q123676955,...,Adolph Damseaux,NaN,NaN,NaN,77398,NaN,NaN,NaN,<NA>,<NA>
30422,K. Tormey,Tormey,K.,2,http://www.botanicalcollections.be/specimen/BR...,"L. J. Dorr, D. Campbell, S. Lampredi & K. Tormey",K. Tormey,Marmey,0.5,http://www.wikidata.org/entity/Q18986812,...,Françoise Marmey,NaN,NaN,NaN,82285,6117-1,Marmey,NaN,<NA>,<NA>


In [22]:
# # TODO further evaluation or filtering, counting, clean up aso.
# if not os.path.exists('data'):
#     os.makedirs('data')
# 
# # meise_collectors_cosine-similarity_wikidata-botanists_%s.csv
# this_output_file='data/results_meise_collectors_vs_wikidata-botanists_cossim-similarity_merged-data_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_wikidata_cossimOrKmeans.to_csv(this_output_file)
# 
# print("Wrote matches of collector names into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )

## Write DarwinCore Attribution Output

Here we map table data fields to fields of DarwinCore Attribution (<https://github.com/tdwg/attribution/>, <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml>) 

### Scoring

Individual scored properties should actually be balanced in such a way that one can simply add up these different property scores; in this case, assessment of the calculated values is still necessary. The problem here with calculation with a distance measure is that we have the opposite of similarity, whose distance can become greater than 1, which must somehow be mapped to a scope of 0 … 1 (or -1 … 0 … 1) (TODO review).

General thoughts: With a score of -1 to 1, it can be assumed that:
* -1 means full devaluation or no agreement
* 1 means full upvoting or agreement, and
* 0 can have several interpretations: it is in between, or no rating possible, or missing values.

### Task to Be Solved in Evaluating the Life Time ~ Rating/Scoring

We have grouped the collection date (evenDate) to the name in the source data, so it may be that for (abbreviated) names, e.g. “Bachmann, F.”, the collection date is valid for *several* personal names, not just one. This must be taken into account when considering and evaluating whether the life data match the collection date. The rating of the life data has the following idea:

| Score (life time) | Remarks | 
|--|--|
| 1.0  | complete match                     |
| 0.5  | somewhat correct, but has errors or mistakes, indicating multiple person names    |
| 0.0   | no evaluation (or not possible) |
| -0.5 | is rather to be rejected, indicating multiple person names and possibly overlapping time spans of the collection date of different person names, or mistakes in the original data |
| -1.0 | completely rejected                |

### Task to Be Solved With Several Names ~ Assessment/Score

Since we do not know if there are other possible names somewhere when there is only one name, we cannot assign a “1” (= full agreement) with certainty, so it was decided that if only 1 name was found, this would be evaluated as zero, in the sense of no evaluation. So when evaluating the multiple names, only the mismatches are evaluated, according to the idea:

| Score (multiple names) | Remarks | 
|--|--|
| 1.0  | this value (=full upvoting or agreement) would never be set in this regard, since we do not know all the full names of the cosmos ;-), and could state this score certainty of 1.0 |
| 0.0 | no evaluation, because only 1 name found | 
| less than 0 | multiple names found, i.e. deduction (perhaps just -0.5, as a decision needs to be made) | 

---

TODO review interpretation:

- the fields are defined in <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml> and regarding from this DwC-attribution concept: is it correct to map it like the following (`name` would represent the *interpreted* resource name (in long format), not the *source* collector `name` (in (theoretically) long format))?
    ```
    name          ← itemLabel (wikiData)
    alternateName ← canonical_string_collector_parsed (actual collector name)
    ```

In [23]:
pprint.pprint(collectors_wikidata_cossimOrKmeans.columns)

Index(['canonical_string_collector_parsed', 'family', 'given',
       'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
       'source_data', 'namematch_source_data', 'namematch_resource_data',
       'namematch_similarity', 'item', 'canonical_string', 'itemLabel',
       'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob',
       'yod'],
      dtype='str')


In [24]:
# refactor collectors_eventDate_mean
# refactor collectors_eventDate_min
# - refactor yob_is_lt_eventDate_min
# refactor collectors_eventDate_max
# - refactr yod_is_gt_eventDate_max
# refactor custom_score_lifetime            → custom_score_lifetime_data
# refactor custom_score_lifetime_annotation → custom_score_lifetime_data_annotation
# refactor namematch_similarity             → namematch_distance
# refactor namematch_similarity_annotation  → namematch_distance_annotation
collectors_wikidata_cossimOrKmeans = collectors_matches_g1_merged_wikidata[
    [
        'canonical_string_collector_parsed', 'family', 'given', 
        'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
        'source_data',
        'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
        'item', 'canonical_string', 'itemLabel',
        'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
        'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max',
        'yob', 'yod',
        # 'wyb'
    ]
].copy()

# order by canonical_string_collector_parsed (actual collector name) (asc)
#   order by similarity (desc) or namematch_distance (asc), 
#     order by number of Wikidata items (asc) and 
#       order by number of collections (desc)
collectors_wikidata_cossimOrKmeans.sort_values(
    by=['canonical_string_collector_parsed', 'namematch_similarity', 'family', 'given'], 
    ascending=[True, False, True, True], 
    inplace=True
)

dwcagent_attr_output=collectors_wikidata_cossimOrKmeans.get([
    "occurrenceID_collectors_firstsample", 
    "canonical_string_collector_parsed",
    'family', 'given',
    "namematch_similarity", 
    "source_data", 
    "itemLabel", 
    "item",
    "collectors_eventDate_min",
    "collectors_eventDate_max",
    'yob', 'yod'
]).copy().drop_duplicates(ignore_index=True)

dwcagent_attr_output['canonical_string_collector_parsed'] = dwcagent_attr_output['canonical_string_collector_parsed'].astype(object)
dwcagent_attr_output['canonical_string_collector_parsed'] = dwcagent_attr_output['canonical_string_collector_parsed'].replace(
    to_replace=r'([^,]+),\s*(.+)',
    value=r'\\2 \\1',
    regex=True
)
dwcagent_attr_output['namematch_similarity_annotation'] = dwcagent_attr_output['namematch_similarity'].astype(str).str.replace(r'(.+)', '\\1 (cos similarity)', regex=True)
# dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'namematch_similarity_annotation', '', allow_duplicates=True)

dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'life_time_periode', '', allow_duplicates=True)

combine_life_times = lambda this_df: ("%s-%s" % (this_df["yob"], this_df["yod"])).replace(r"<NA>", "?")
dwcagent_attr_output["life_time_periode"]=dwcagent_attr_output.apply(combine_life_times, axis="columns")

# dwcagent_attr_output["life_time_periode"]

years_from_birth_until_first_collection_activity = 10
dwcagent_attr_output["yob_is_lt_eventDate_min"] = dwcagent_attr_output["yob"] + years_from_birth_until_first_collection_activity < dwcagent_attr_output["collectors_eventDate_min"].dt.year
dwcagent_attr_output["yod_is_gt_eventDate_max"] = dwcagent_attr_output["yod"] > dwcagent_attr_output["collectors_eventDate_max"].dt.year
dwcagent_attr_output["custom_score_lifetime_data"] = 0.0
dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'custom_score_lifetime_data_annotation', '', allow_duplicates=True)

# df.loc[(df['column_of_interest'] … condition), 'fill_to_column'] = value 

dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime_data"
] = 1.0
# True cases but <NA> missing values
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime_data"
] = 1.0
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime_data"
] = 1.0
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime_data"
] = 0.0

# False cases
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime_data"
] = -1.0
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==True) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime_data"
] = 0.5
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == True),
    "custom_score_lifetime_data"
] = 0.5

# False cases but <NA> missing values
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull()),
    "custom_score_lifetime_data"
] = -0.5
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull()) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime_data"
] = -0.5

# annotations True cases
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_data_annotation"
] = "full match"

# annotations True cases but <NA> missing values
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_data_annotation"
] = "OK? year of death is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_data_annotation"
] = "OK? year of birth is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_data_annotation"
] = "unknown life time"

# annotations False cases
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_data_annotation"
] = "life time not matching any enventDate (yob + %s … yod)" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==True) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_data_annotation"
] = "OK yob + %s, but yod not matching, check name and liftime data" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == True), 
    "custom_score_lifetime_data_annotation"
] = "yob + %s not matching, OK yod, check name and liftime data" % years_from_birth_until_first_collection_activity
# annotations False cases but <NA> missing values
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull()), 
    "custom_score_lifetime_data_annotation"
] = "yob + %s not matching, yod unknown, check name and liftime data" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull()) & (dwcagent_attr_output["yod_is_gt_eventDate_max"]==False), 
    "custom_score_lifetime_data_annotation"
] = "yob unknown, yod not matching, check name and liftime data"

dwcagent_attr_output["custom_score_multiple_names"]=0.0 # 0 shall mean: we don’t know yet for real
dwcagent_attr_output.loc[
    (dwcagent_attr_output['canonical_string_collector_parsed'].duplicated(keep=False)),
    'custom_score_multiple_names'
] = -0.5 # one decision has to be made, so cut the range of -1 to 0 only into half (or include multiple count somehow?)

dwcagent_attr_output['custom_score_overall'] = (
    dwcagent_attr_output['namematch_similarity'] * \
    (
        ( dwcagent_attr_output["custom_score_lifetime_data"] + dwcagent_attr_output['custom_score_multiple_names']) / 2
    )
).round(3)

dwcagent_attr_output['attributionRemarks'] = dwcagent_attr_output.apply(
    lambda row: "{similarity_distance_note};"
                " {score_overall:.2f} (score overall);"
                " {lifetime_periode} (life time);"
                " {lifetime_score:.1f} (life time score);"
                " {lifetime_score_annote} (life time score note);"
                " {score_multinames:.2f} (score multiple names);"
        .format(
    similarity_distance_note=row['namematch_similarity_annotation'],
    lifetime_periode=row["life_time_periode"],
    lifetime_score=row["custom_score_lifetime_data"],
    lifetime_score_annote=row["custom_score_lifetime_data_annotation"],
    score_overall=row["custom_score_overall"],
    score_multinames=row["custom_score_multiple_names"]
    ), axis='columns'
)

# adjust dwcagent displayOrder also to olerall score
dwcagent_attr_output.sort_values(
    # by=['namematch_similarity', 'family', 'given', 'custom_score_overall'], 
    # ascending=[False, True, True, False], 
    by=['canonical_string_collector_parsed', 'custom_score_overall', 'family', 'given'], 
    ascending=[True, False, True, True], 
    inplace=True
)
# use ordered canonical_string_collector_parsed to generate displayOrder
temp_duplicated = dwcagent_attr_output['canonical_string_collector_parsed'].duplicated() 
    # duplicated() keeps the first value False and mark all other duplicats as True, i.e. we can cumulate the Trues, it gives the order index
temp_insert_value=temp_duplicated.groupby(dwcagent_attr_output['canonical_string_collector_parsed']).cumsum() + 1 # display order starts at 1, incrementing
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('canonical_string_collector_parsed') + 1, 'displayOrder', temp_insert_value, allow_duplicates=True)

# test an show example data
if explain_and_show_the_data:
    print("example data: names having year of birth (yob) < eventDate_min")
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == True].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))
    print("example data: names having year of birth (yob) > eventDate_min")
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == False].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))

example data: names having year of birth (yob) < eventDate_min


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime_data,custom_score_lifetime_data_annotation
7,A'd,Andrew Adie Dalglish,0.404,0.808 (cos similarity); 0.40 (score overall); ...,0.0,0.808,1868-1924,1896-08-06,1896-08-06,True,True,1.0,full match
8,A-J,Alice J. Heading,0.186,0.745 (cos similarity); 0.19 (score overall); ...,0.0,0.745,1852-1945,1948-07-01,1948-07-01,True,False,0.5,"OK yob + 10, but yod not matching, check name ..."
13,A. A. Sagástegui,Abundio Sagástegui Alva,0.487,0.974 (cos similarity); 0.49 (score overall); ...,0.0,0.974,1932-2012,1981-09-13,1986-07-12,True,True,1.0,full match
18,A. Acebey,Amparo Acebey,0.500,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.000,1973-?,2000-11-17,2000-11-17,True,<NA>,1.0,OK? year of death is missing
22,A. Aguayo,Carlos Guillermo Aguayo,0.214,0.857 (cos similarity); 0.21 (score overall); ...,0.0,0.857,1899-1982,1989-01-26,1989-10-31,True,False,0.5,"OK yob + 10, but yod not matching, check name ..."


example data: names having year of birth (yob) > eventDate_min


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime_data,custom_score_lifetime_data_annotation
0,A,Anna Atkins,-0.365,0.73 (cos similarity); -0.36 (score overall); ...,0.0,0.730,1799-1871,1809-01-01,2000-07-14,False,False,-1.0,life time not matching any enventDate (yob + 1...
1,A Funk,Alvin Funk,0.250,1.0 (cos similarity); 0.25 (score overall); 19...,0.0,1.000,1925-2010,NaT,NaT,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."
2,A Isaac Holden,Frances Margaret Leighton,0.177,0.708 (cos similarity); 0.18 (score overall); ...,0.0,0.708,1909-2006,1892-09-01,1892-09-01,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."
3,A Jablonski,Eugene Jablonszky,0.225,0.901 (cos similarity); 0.23 (score overall); ...,0.0,0.901,1892-1975,NaT,NaT,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."
6,A Kühlewein,Alexander Eberhard von Kühlewein,0.223,0.892 (cos similarity); 0.22 (score overall); ...,0.0,0.892,1791-1888,NaT,NaT,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."


In [25]:
column_map_dwcagent_attr = {
    'occurrenceID_collectors_firstsample':'occurrenceID',
    'canonical_string_collector_parsed':  'alternateName',
    'source_data':                        'verbatimName',
    'itemLabel':                          'name',
    'item':                               'identifier',
    'collectors_eventDate_min':           'startedAtTime',
    'collectors_eventDate_max':           'endedAtTime',
    'namematch_similarity':               'custom_namematch_similarity'
}
dwcagent_attr_output.rename(
    mapper=column_map_dwcagent_attr,
    axis='columns',
    inplace=True)

dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'agentIdentifierType', 'wikidata' , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('agentIdentifierType') + 1, 'agentType'          , 'Person'   , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'action'             , 'collected', allow_duplicates=True)

if explain_and_show_the_data:
    print("the mapped DarwinCore attribution output examples, sorted by alternateName (=collector name) + displayOrder …")
    display(dwcagent_attr_output.head(20))

dwcagent_attr_output=dwcagent_attr_output.reindex(
    columns=[
        'occurrenceID', # no DwC agent standard (yet)?
        'verbatimName',
        'alternateName',
        'displayOrder', # shall start from 1, 2, 3 …
        'name',
        'attributionRemarks',
        'startedAtTime',
        'endedAtTime',
        'agentType',
        'action',
        'agentIdentifierType',
        'identifier',
        "custom_score_overall", # keep it for calculation convenience, no standard in DwC agent
        'custom_namematch_similarity',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_multiple_names',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_lifetime_data' # keep it for calculation convenience, no standard in DwC agent
    ]
)
# column deletion not neccessary after ….reindex(columns=[…])
# for this_column in ['yob', 'yod', 'life_time_periode', 'yob_is_lt_eventDate_min', 'yod_is_gt_eventDate_max', 'score_lifetime_annotation']:
#     del dwcagent_attr_output[this_column]


the mapped DarwinCore attribution output examples, sorted by alternateName (=collector name) + displayOrder …


,occurrenceID,alternateName,displayOrder,family,given,custom_namematch_similarity,verbatimName,name,identifier,action,...,yod,namematch_similarity_annotation,life_time_periode,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime_data,custom_score_lifetime_data_annotation,custom_score_multiple_names,custom_score_overall,attributionRemarks
0,http://www.botanicalcollections.be/specimen/BR...,A,1,A,NaN,0.730,"Church, A.C., Ismail, Ruskandi, A",Anna Atkins,http://www.wikidata.org/entity/Q264269,collected,...,1871,0.73 (cos similarity),1799-1871,False,False,-1.0,life time not matching any enventDate (yob + 1...,0.0,-0.365,0.73 (cos similarity); -0.36 (score overall); ...
1,http://www.botanicalcollections.be/specimen/BR...,A Funk,1,Funk,A,1.000,a Funk,Alvin Funk,http://www.wikidata.org/entity/Q21337490,collected,...,2010,1.0 (cos similarity),1925-2010,False,True,0.5,"yob + 10 not matching, OK yod, check name and ...",0.0,0.250,1.0 (cos similarity); 0.25 (score overall); 19...
2,http://www.botanicalcollections.be/specimen/BR...,A Isaac Holden,1,Isaac Holden,A,0.708,a) Isaac Holden;b) W.A. Setchell,Frances Margaret Leighton,http://www.wikidata.org/entity/Q5864685,collected,...,2006,0.708 (cos similarity),1909-2006,False,True,0.5,"yob + 10 not matching, OK yod, check name and ...",0.0,0.177,0.708 (cos similarity); 0.18 (score overall); ...
3,http://www.botanicalcollections.be/specimen/BR...,A Jablonski,1,Jablonski,A,0.901,a Jablonski,Eugene Jablonszky,http://www.wikidata.org/entity/Q3059563,collected,...,1975,0.901 (cos similarity),1892-1975,False,True,0.5,"yob + 10 not matching, OK yod, check name and ...",0.0,0.225,0.901 (cos similarity); 0.23 (score overall); ...
4,http://www.botanicalcollections.be/specimen/BR...,A Kuhlew,1,Kuhlew,A,0.625,Dr. a Kuhlew.,Ralph Kuhlenkamp,http://www.wikidata.org/entity/Q36665275,collected,...,<NA>,0.625 (cos similarity),?-?,<NA>,<NA>,0.0,unknown life time,0.0,0.000,0.625 (cos similarity); 0.00 (score overall); ...
5,http://www.botanicalcollections.be/specimen/BR...,A Kuhlewein,1,Kuhlewein,A,0.542,Dr. a Kuhlewein,Ralph Kuhlenkamp,http://www.wikidata.org/entity/Q36665275,collected,...,<NA>,0.542 (cos similarity),?-?,<NA>,<NA>,0.0,unknown life time,0.0,0.000,0.542 (cos similarity); 0.00 (score overall); ...
6,http://www.botanicalcollections.be/specimen/BR...,A Kühlewein,1,Kühlewein,A,0.892,Dr. a Kühlewein,Alexander Eberhard von Kühlewein,http://www.wikidata.org/entity/Q118869465,collected,...,1888,0.892 (cos similarity),1791-1888,False,True,0.5,"yob + 10 not matching, OK yod, check name and ...",0.0,0.223,0.892 (cos similarity); 0.22 (score overall); ...
7,http://www.botanicalcollections.be/specimen/BR...,A'd,1,A'd,NaN,0.808,A'[D],Andrew Adie Dalglish,http://www.wikidata.org/entity/Q114055421,collected,...,1924,0.808 (cos similarity),1868-1924,True,True,1.0,full match,0.0,0.404,0.808 (cos similarity); 0.40 (score overall); ...
8,http://www.botanicalcollections.be/specimen/BR...,A-J,1,A-J,NaN,0.745,A-J & Christ. Bange,Alice J. Heading,http://www.wikidata.org/entity/Q97940058,collected,...,1945,0.745 (cos similarity),1852-1945,True,False,0.5,"OK yob + 10, but yod not matching, check name ...",0.0,0.186,0.745 (cos similarity); 0.19 (score overall); ...
9,http://www.botanicalcollections.be/specimen/BR...,A. A. Bauy Braun,1,A. Bauy Braun,A.,0.629,A. B[auy] [= Braun A.?],Alexander Braun,http://www.wikidata.org/entity/Q62855,collected,...,1877,0.629 (cos similarity),1805-1877,False,True,0.5,"yob + 10 not matching, OK yod, check name and ...",0.0,0.157,0.629 (cos similarity); 0.16 (score overall); ...


In [26]:
if explain_and_show_the_data:
    print("show column-reduced examples of ?multiple name cases …")
    # criterion = dwcagent_attr_output['alternateName'].map(lambda x: x.startswith('S. Ahmad'))
    criterion = dwcagent_attr_output['custom_score_multiple_names'].map(lambda this_score: this_score < 0 )  # show multiple name cases
    
    # display(dwcagent_attr_output[criterion].head(20))
    display(dwcagent_attr_output[criterion].drop(['agentType', 'action', 'agentIdentifierType'], axis='columns').head(20))

show column-reduced examples of ?multiple name cases …


,occurrenceID,verbatimName,alternateName,displayOrder,name,attributionRemarks,startedAtTime,endedAtTime,identifier,custom_score_overall,custom_namematch_similarity,custom_score_multiple_names,custom_score_lifetime_data
10,http://www.botanicalcollections.be/specimen/BR...,"Y.S. Abeid, A. Hernández A., H. Katandasha & B...",A. A. Hernández,1,Antonio Hernández,0.956 (cos similarity); -0.24 (score overall);...,2002-12-17,2002-12-17,http://www.wikidata.org/entity/Q21516463,-0.239,0.956,-0.5,0.0
11,http://www.botanicalcollections.be/specimen/BR...,"Y.S. Abeid, A. Hernández A., H. Katandasha & B...",A. A. Hernández,2,Alexandra Hernández,0.956 (cos similarity); -0.24 (score overall);...,2002-12-17,2002-12-17,http://www.wikidata.org/entity/Q36503191,-0.239,0.956,-0.5,0.0
49,http://www.botanicalcollections.be/specimen/BR...,"A. Andersson, J. Lå[uzzat]",A. Andersson,1,Axel Andersson,1.0 (cos similarity); -0.50 (score overall); 1...,1869-06-23,1981-08-23,http://www.wikidata.org/entity/Q123652899,-0.500,1.000,-0.5,-0.5
51,http://www.botanicalcollections.be/specimen/BR...,"A. Andersson, J. Lå[uzzat]",A. Andersson,2,I. Anita Andersson,1.0 (cos similarity); -0.50 (score overall); 1...,1869-06-23,1981-08-23,http://www.wikidata.org/entity/Q21505194,-0.500,1.000,-0.5,-0.5
50,http://www.botanicalcollections.be/specimen/BR...,"A. Andersson, J. Lå[uzzat]",A. Andersson,3,Karl Alfred Andersson,1.0 (cos similarity); -0.75 (score overall); 1...,1869-06-23,1981-08-23,http://www.wikidata.org/entity/Q131724787,-0.750,1.000,-0.5,-1.0
84,http://www.botanicalcollections.be/specimen/BR...,Aubert A.,A. Aubert,1,Gustave Aubert,0.927 (cos similarity); 0.00 (score overall); ...,NaT,NaT,http://www.wikidata.org/entity/Q21505424,0.000,0.927,-0.5,0.5
82,http://www.botanicalcollections.be/specimen/BR...,Aubert A.,A. Aubert,2,Pierre-Léonce Aubert,0.927 (cos similarity); -0.23 (score overall);...,NaT,NaT,http://www.wikidata.org/entity/Q123676187,-0.232,0.927,-0.5,0.0
83,http://www.botanicalcollections.be/specimen/BR...,Aubert A.,A. Aubert,3,Samuel Aubert,0.927 (cos similarity); -0.23 (score overall);...,NaT,NaT,http://www.wikidata.org/entity/Q123676188,-0.232,0.927,-0.5,0.0
102,http://www.botanicalcollections.be/specimen/BR...,"Brooks R.R., Baker A., Schat H. & Malaisse F.",A. Baker,1,Alan N. Baker,1.0 (cos similarity); 0.25 (score overall); 19...,1990-01-01,1990-06-02,http://www.wikidata.org/entity/Q22105323,0.250,1.000,-0.5,1.0
101,http://www.botanicalcollections.be/specimen/BR...,"Brooks R.R., Baker A., Schat H. & Malaisse F.",A. Baker,2,Arthur J. D. Baker,1.0 (cos similarity); -0.25 (score overall); ?...,1990-01-01,1990-06-02,http://www.wikidata.org/entity/Q114860418,-0.250,1.000,-0.5,0.0


In [27]:
# TODO further evaluation or filtering, counting, clean up aso.
if not os.path.exists('data'):
    os.makedirs('data')

# this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
# this_timestamp_for_data=20231116
this_output_file='data/results_meise_collectors-eventDate_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_%s.csv' % (
    this_timestamp_for_data
)

dwcagent_attr_output.to_csv(this_output_file, index=False)

print("Wrote matches of collector names as dwc-agent-output into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote matches of collector names as dwc-agent-output into data/results_meise_collectors-eventDate_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_20260210.csv (22510 kB)


## Documentation

Explanation of columns (TODO review documentation):

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
occurrenceID_collectors_count | count of all occurrenceID of one particular collector name
occurrenceID_collectors_firstsample | a data sample of an occurrenceID 
eventDate | date of the sampling event (required by GBIF, ☞ https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
namematch_source_data | matched name of the collector data set
namematch_resource_data | matched name of Wikidata the collector was tried to matched to
namematch_similarity | calculated cosine-similarity
**DarwinCore Agent Output** | (☞ [agent_actions_v2020-09-08.xml](https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml))
occurrenceID | occurrence ID of the data item
name | the interpreted name match (https://github.com/tdwg/attribution/ The name of the item. In this case the *full name* as would be written on a legal document (without abbreviation), eg givenName familyName)
verbatimName | the source data name(s) (https://github.com/tdwg/attribution/ As written on occurrence, such as the collection or determination label.)
alternateName | the input name, collector source name (An alias for the item. Other full name agent may have been known under such as maiden name.)
displayOrder | I guess ordering the multiple name cases (https://github.com/tdwg/attribution/ The display order for the agent that executed the action when more than one agent was a participant.)
attributionRemarks | notes on the results (distance or similarity), including calculated value
agentType | The nature of the agent, e.g. "Person", "Organization", "SoftwareApplication"
action | The name of the single action written as a verb in past tense. Recommended best practice is to use a controlled vocabulary, examples "collected" or "identified"
agentIdentifierType | The type of identifier for the agent. (https://github.com/tdwg/attribution/ Recommended best practice is to use a controlled vocabulary, e.g. “ORCID”, “ISNI”, “Wikidata”, “VIAF”, “RoR”, “Ringgold”, “GRID”).
identifier | Wikidata ID (Recommended practice is to identify the resource by means of a string conforming to an identification system. Examples include International Standard Book Number (ISBN), Digital Object Identifier (DOI), and Uniform Resource Name (URN). Persistent identifiers should be provided as HTTP URIs.)
startedAtTime | the first date of eventDate (supposedly the first sampling date), but grouped from collector name (https://github.com/tdwg/attribution/ Start is when an action is deemed to have been started by an agent.)—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
endedAtTime | the last date of eventDate (supposedly the last sampling date), but grouped from collector name (https://github.com/tdwg/attribution/ End is when an action is deemed to have been ended by an agent.)—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))